In [1]:
import pandas as pd
import os
import re
from nltk.corpus import stopwords
import nltk.data
import logging
import numpy as np  # Make sure that numpy is imported
from gensim.models import Word2Vec
from sklearn.ensemble import RandomForestClassifier
from nltk.corpus import stopwords
from bs4 import BeautifulSoup

# Download the punkt tokenizer for sentence splitting
import nltk.data


In [2]:
labeled = pd.read_csv("labeledTrainData.tsv", header=0, delimiter="\t", quoting=2)
test = pd.read_csv("testData.tsv", header=0, delimiter="\t", quoting=2)
unlabeled = pd.read_csv("unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

In [21]:
texts = list(labeled.review) + list(test.review) + list(unlabeled.review)

In [22]:
len(texts)

100000

In [10]:
def text_to_wordlist(text):
    text = re.sub('n\'t', ' not', text)
    
    text = re.sub('[^a-zA-Z]', ' ', text)
    words = text.lower().split()

    return words

In [14]:
import nltk.data
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

def text_to_sentences(text):
    text = BeautifulSoup(text).get_text()
    
    raw_sentences = tokenizer.tokenize(text.strip())
    
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(text_to_wordlist(raw_sentence))

    return sentences

In [23]:
%%time
sentences = []

for review in texts:
    sentences += text_to_sentences(review)

CPU times: user 2min 5s, sys: 4.22 s, total: 2min 9s
Wall time: 2min 14s


In [24]:
len(sentences)

1059164

In [25]:
num_features = 300  # итоговая размерность вектора каждого слова
min_word_count = 5  # минимальная частотность слова, чтобы оно попало в модель
num_workers = 2     # количество ядер вашего процессора, чтоб запустить обучение в несколько потоков
context = 10        # размер окна 
downsampling = 1e-3 # внутренняя метрика модели

model = Word2Vec(sentences, workers=num_workers, size=num_features,
                 min_count=min_word_count, window=context, sample=downsampling)

In [26]:
# Финализируем нашу модель. Ее нельзя будет доучить теперь, но она начнет занимать гораздо меньше места
model.init_sims(replace=True)

In [37]:
model.most_similar('vodka')

[(u'beer', 0.6807037591934204),
 (u'champagne', 0.6553428769111633),
 (u'coffee', 0.6497250199317932),
 (u'puking', 0.641952395439148),
 (u'sipping', 0.6341596245765686),
 (u'booze', 0.627311110496521),
 (u'smoking', 0.6170046925544739),
 (u'drinking', 0.6128978729248047),
 (u'cigarettes', 0.6115179061889648),
 (u'cigars', 0.6113725900650024)]

In [47]:
model.doesnt_match(['berlin', 'moscow', 'kz', 'riga', 'utah'])

'berlin'

In [50]:
len(model.index2word)

53171

In [51]:
import numpy as np

def text_to_vec(words, model, size):
    text_vec = np.zeros((size,), dtype="float32")
    n_words = 0

    index2word_set = set(model.index2word)
    for word in words:
        if word in index2word_set:
            n_words = n_words + 1
            text_vec = np.add(text_vec, model[word])
    
    if n_words != 0:
        text_vec /= n_words
    return text_vec


In [55]:
def texts_to_vecs(texts, model, size):
    texts_vecs = np.zeros((len(texts), size), dtype="float32")
    
    for i, text in enumerate(texts):
        texts_vecs[i] = text_to_vec(text, model, size)

    return texts_vecs


In [53]:
train_like_word_list = \
[sum(text_to_sentences(text), []) for text in labeled.review]

In [57]:
train_vecs = texts_to_vecs(train_like_word_list, model, num_features)

In [58]:
test_like_word_list = \
[sum(text_to_sentences(text), []) for text in test.review]
test_vecs = texts_to_vecs(test_like_word_list, model, num_features)

In [60]:
from sklearn.ensemble import RandomForestClassifier

forest = RandomForestClassifier(n_estimators=100, n_jobs=8)
forest = forest.fit(train_vecs, labeled["sentiment"])
predict = forest.predict(test_vecs)

In [62]:
len(predict)

25000

In [63]:
%%time
# Кластеризируем все слова. 

from sklearn.cluster import KMeans

word_vectors = model.syn0
# Число кластеров установим в 10000. Для этого числа нет "серебряной пули". Для каждого случая лучше подойдет разная
num_clusters = 1000

# Начнем кластеризацию, учитывая что классов много, количество векторов (по сути слов) много,
#   все это будет происходит продолжительное время. Можно сходить за чаем.
kmeans_clustering = KMeans(n_clusters=num_clusters)
idx = kmeans_clustering.fit_predict(word_vectors)

# в idx будут храниться номера классов для каждого слова

CPU times: user 48min 41s, sys: 33.3 s, total: 49min 15s
Wall time: 1h 41min 31s
